In [16]:
# Importing required libraries.

import cv2   #For Image Processing

import time  # To make the system sleep before video capturing starts

import numpy as np  # To operate on various functionalities

In [18]:
# The output video after becoming invisible should be saved. So creating an output object.
# To save a video , VideoWriter class object is created.

# SYNTAX: 
# out = cv2.VideoWriter(<< Location>>, << Four CC Code>>, << No. of frames per second>>, << size of the video>>)

# Location: Where to save the video.
# Size: tuple(height, width of the frame)
# fourcc code: 4 byte code which depicts the video codec.

fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter('magic.avi',fourcc,20.0,(640,480))

In [19]:
# Reading video '0' for webcam else route to video can be given.

cap = cv2.VideoCapture(0)

#Allow the system to sleep for 3 sec before webcam starts (To capture the background image)

time.sleep(3)
count = 0
background = 0

#Capturing the background

for i in range(60):
    temp,background = cap.read()

background = np.flip(background, axis=1)

# Capture the real time frame from webcam

while(cap.isOpened()):

    temp, img = cap.read()
    if not temp:
        break
    count += 1
    img = np.flip(img, axis=1)

    # Since BGR images are more sensitive to the light, so they needs to be converted to HSV.
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Blue color clock is to be considered.
    lower_range = np.array([110,50,50])
    higher_range = np.array([130,255,255])

    mask1 = cv2.inRange(hsv, lower_range, higher_range)

    lower_range = np.array([170,120,70])
    higher_range = np.array([180,255,255])

    mask2 = cv2.inRange(hsv, lower_range, higher_range)

    mask1 = mask1 + mask2

    # Morphological operations performed on the image.
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

    # Filtering out cloth color(Blue color)

    mask2 = cv2.bitwise_not(mask1)

    ## Segment the Blue color part out of the frame using bitwise and with the inverted mask
    layer1 = cv2.bitwise_and(img, img, mask = mask2)

    ## Create image showing static background frame pixels only for the masked region
    layer2 = cv2.bitwise_and(background, background, mask = mask1)

    #output
    output = cv2.addWeighted(layer1,1,layer2,1,0)
    out.write(output)
    #cv2.putText(output, "To stop press 'q'", (100,15), cv2.FONT_HERSHEY_SIMPLEX ,0.6 , (0,), 3)
    cv2.imshow("Output",output)

    k = cv2.waitKey(3) & 0xFF
    if k == ord('q') :
        cap.release()
        out.release()
        cv2.destroyAllWindows()